In [1]:
#export
import logging
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms , utils
from torch.utils.data import Dataset , DataLoader
import time
from src import DenseNet
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
#export
def train_and_validate(model,
          epochs,
          lr,
          weight_decay,
          trainloader,
          validloader,
          criterion):
    
    best_loss = 999999
    start = time.time()
    
    for epoch in range(1,epochs+1):
        # setting : train
        model.train()
        running_loss = 0.0
        total_done = 0
        print(f'Epoch {epoch}')
        optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                                      model.parameters()),
                               lr = lr,
                               weight_decay = weight_decay)
        print('-'*50)
        for i , (inputs,labels) in enumerate(trainloader):
            batch_size = inputs.shape[0]
            inputs , labels =  inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * batch_size
            print(f'Batch: {i+1/batch_size}')
        epoch_loss = running_loss / len(trainloader.dataset)
        print(f'epoch_train_loss: {".4f"%epoch_loss }')
        # Learning rate decay
        if epoch % 30 == 0:
            print(f'Learning rate changed from {lr} to {lr/10}')
            lr = lr/10
            optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                                          model.parameters()),
                                   lr = lr,
                                   weight_decay = weight_decay)
        # setting : valid
        valid_loss = 0
        model.eval()
        for i , (inputs,labels) in enumerate(validloader):
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            batch_size = inputs.shape[0]
            valid_loss += loss.item() * batch_size 
        print(f'valid_loss : {valid_loss /len(validloader.dataset)}, Accuracy: {(torch.argmax(outputs,-1)==labels).sum().item()}')
        print('-'*50)
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            print('Saving...')
            state = {
                'model' : model,
                'best_loss': best_loss,
                'epoch' : epoch,
                'rng_state' : torch.get_rng_state(),
                'lr' : lr
            }
            torch.save(state , 'results/checkpoint')
            
    end = time.time()
    elapsed = end - start
    print(f'Time Elapsed : {elapsed}')
    total_done += batch_size
    best_state = torch.load('results/checkpoint')
    model = best_state['model']
        
    return model , best_epoch , valid_loss 

In [4]:
#export
def learner(train_loader,
            val_loader,
            lr,
            weight_decay,
            incl_layers,
            freeze_layers,
            drop_rate,
            epochs,
            return_preds=False):
    
    if not torch.cuda.is_available():
        gpu = True
        raise ValueError('GPU is necessary!')
        
    print(f'Learning rate : {lr}')
    print(f'Drop rate : {drop_rate}')
    print(f'Included Layers in the blocks: {inc_layers}')
    
    model = DenseNet.densenet121(pretrained=True,num_layers=incl_layers,drop_rate=drop_rate)
    
    print(f'Freezed layers in the blocks : {freeze_layers}') 
    
    if freeze_layers > 42:
        freeze_trans = 3 
    elif freeze_layers > 18:
        freeze_trans = 2
    elif freeze_layers > 6:
        freeze_trans = 1
    elif freeze_layers >=0:
        freeze_trans = 0
    
    i = 0
    freeze_count = 0
    # FOR EACH DENSE LAYER (6 parameter tensors)
    # norm1.w , norm1.b , conv1.w , norm2.w , norm2.b ,conv2.w
    freeze_count += freeze_layers * 6
    # FOR EACH TRANSITION LAYER (3 parameter tensors)
    # norm.w , norm.b , conv.w
    freeze_count += freeze_trans * 3
    # FOR FIRST CONV LAYER (3 parameter tensors)
    # conv0.w , norm0.w , norm0.b
    if freeze_count > 0:
        freeze_count += 3
    print(f'Freeze count {freeze_count}')
    for name , param in model.named_parameters():
        if i < freeze_count:
            if 'sEBlock' in name:
                print(f'Passing Over {name}')
            else:
                param.requires_grad = False
                i += 1
    
    criterion = nn.CrossEntropyLoss()    
    model , best_epoch , valid_loss = train_and_validate(model,
                                                         epochs,
                                                         lr,
                                                         weight_decay,
                                                         trainloader,
                                                         validloader,
                                                         criterion)
    if return_preds:
        for inputs , labels in validloader:
            outputs = model(inputs)
            preds = torch.argmax(outputs,-1)
            return valid_loss , preds
    else:
        return valid_loss

SyntaxError: invalid syntax (<ipython-input-4-9988fff06b20>, line 10)

In [5]:
!python notebook2script.py Train.ipynb

Converted Train.ipynb to src\Train.py


In [2]:
torch.set_default_tensor_type??